<a href="https://colab.research.google.com/github/ahliminha/P300/blob/master/P300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install mat4py

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy.io as sio
from mat4py import loadmat 
from keras.utils import to_categorical
np.random.seed(816)

Using TensorFlow backend.


In [0]:
import glob

In [5]:
folder = '/content/drive/My Drive/IC/*.mat'
files = glob.glob(folder)
files.pop(0)
files.pop(0)

'/content/drive/My Drive/IC/P300S02.mat'

In [6]:
print(files)

['/content/drive/My Drive/IC/P300S03.mat', '/content/drive/My Drive/IC/P300S04.mat', '/content/drive/My Drive/IC/P300S05.mat', '/content/drive/My Drive/IC/P300S06.mat', '/content/drive/My Drive/IC/P300S07.mat', '/content/drive/My Drive/IC/P300S08.mat']


In [0]:
from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Filter requirements.
order = 6
fs = 24.0       # sample rate, Hz
cutoff = 1.5  # desired cutoff frequency of the filter, Hz

# Get the filter coefficients so we can check its frequency response.
b, a = butter_lowpass(cutoff, fs, order)

In [0]:
def prepare_data(file):
  allX = []
  allY = []
  allT = []
  allF = []
  raw_data = loadmat(file)
  useful_data = raw_data['data'].copy()
  X = useful_data['X']
  Y = useful_data['y']
  T = useful_data['trial']
  F = useful_data['flash']
  X_mean = np.mean(X, axis=1)
  X_filtered = butter_lowpass_filter(X_mean, cutoff, fs, order)
  
  return X_filtered, Y, T, F

In [0]:
def normalized(vec):
  norm_vec = (vec - vec.min(axis=1, keepdims=True))/vec.ptp(axis=1, keepdims=True)
  return norm_vec

In [0]:
def clean_data(X, Y, flash):
  print(flash)
  X_samples = np.array([np.array(X[i[0]:i[0]+300]) for i in flash] )
  column    = [i[2] for i in flash]
  label     = [i[3] - 1 for i in flash]
  
  LIMIT = 4080
  X_selected = np.array(X_samples[:LIMIT])
  col_selected = np.array(column[:LIMIT])
  label_selected = np.array(label[:LIMIT])

  y = np.array(to_categorical(label_selected))

  X_timeseries = np.vstack(X_selected)
  X_letters = X_timeseries.reshape(34,120,300,1)
  y_letters = y.reshape(34,120,2)
  cleaned_X = np.vstack(X_letters)
  cleaned_Y = np.vstack(y_letters)


  return cleaned_X, cleaned_Y

In [11]:
import tensorflow as tf

from tensorflow import keras
tf.keras.backend.clear_session()  # For easy reset of notebook state.

from tensorflow.keras import layers

input_layer = keras.Input(shape = (300,1), name='main_input')
#lstm  = layers.Bidirectional(layers.LSTM(64, return_sequences=True))
#lstm1 = lstm(input_layer)
x     = layers.Dense(8)(input_layer)
x     = layers.Dense(16)(x)
x     = layers.Dense(8)(x)
x     = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs=input_layer, outputs=output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 300, 1)]          0         
_________________________________________________________________
dense (Dense)                (None, 300, 8)            16        
_________________________________________________________________
dense_1 (Dense)              (None, 300, 16)           144       
_________________________________________________________________
dense_2 (Dense)              (None, 300, 8)            136       
_________________________________________________________________
global_average_pooling1d (Gl (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total params: 314
Trainable params: 314
Non-trainable params: 0
_______________________________________________________________

In [0]:
#compiling the model
opt = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.9, amsgrad=True)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [13]:
for file in files:
  print(file)
  X, Y, Trials, Flash = prepare_data(file)
  X, y = clean_data(X, Y, Flash)
  history = model.fit(X, y, epochs=10, batch_size=10, verbose=1)

/content/drive/My Drive/IC/P300S03.mat
[[7481, 30, 11, 1], [7540, 32, 3, 1], [7603, 32, 12, 1], [7665, 32, 2, 1], [7728, 33, 8, 2], [7792, 33, 6, 1], [7856, 33, 7, 1], [7920, 32, 1, 1], [7984, 32, 9, 1], [8046, 32, 5, 1], [8108, 32, 10, 1], [8171, 32, 4, 2], [8256, 14, 10, 1], [8300, 32, 1, 1], [8363, 32, 9, 1], [8425, 32, 2, 1], [8487, 32, 12, 1], [8550, 32, 5, 1], [8610, 31, 7, 1], [8672, 32, 3, 1], [8736, 31, 8, 2], [8736, 30, 4, 2], [8798, 32, 11, 1], [8922, 32, 6, 1], [8993, 26, 7, 1], [9049, 31, 1, 1], [9110, 32, 8, 2], [9174, 32, 4, 2], [9237, 31, 9, 1], [9297, 32, 6, 1], [9361, 32, 10, 1], [9361, 32, 3, 1], [9487, 29, 11, 1], [9487, 32, 2, 1], [9612, 32, 12, 1], [9675, 32, 5, 1], [9744, 26, 12, 1], [9800, 32, 2, 1], [9864, 32, 9, 1], [9927, 29, 6, 1], [9988, 32, 8, 2], [9988, 32, 3, 1], [10115, 32, 7, 1], [10175, 31, 4, 2], [10238, 32, 11, 1], [10301, 32, 1, 1], [10364, 30, 10, 1], [10424, 32, 5, 1], [10508, 15, 8, 2], [10554, 32, 2, 1], [10616, 31, 7, 1], [10678, 32, 3, 1], [1

In [0]:
plt.style.use('ggplot')

In [0]:
plt.plot(X_filtered[trial[5]:trial[5]+351])

In [0]:
plt.plot(X_filtered[trial[0]:trial[0]+351])

In [0]:
def Mean_Over_Flash(vector, n_cols, n_excitments):
  final_vec = []
  vec_size = vector.shape[0]
  for vec in vector:
    new_vec = vec.reshape(n_cols, n_excitments, 351, 1)
    MOF = np.mean(new_vec, 1)
    final_vec.append(MOF)
  final_vec = np.asarray(final_vec)
  return final_vec.reshape(vec_size*n_cols, 351, 1)

In [0]:
X_tr = Mean_Over_Flash(X_train, 12, 10)
X_te = Mean_Over_Flash(X_test, 12, 10)

In [0]:
plt.plot(X_tr[4])
plt.plot(X_tr[0])
plt.draw()

In [0]:
def Adjust_Label_Dims(label_vec, n_excitments):
  new_label = []
  for step in label_vec:
    new_label.append(step[:12])
  new_label = np.asarray(new_label)
  return new_label.reshape(n_excitments*12, 2)

In [0]:
score = model.evaluate(X_te, y_te, batch_size=1, verbose=1)

In [0]:
print(score)

In [0]:
pred = model.predict(X_te)

In [0]:
for k,i in enumerate(label[1800:1805]):
  if i == 2:  
    print(k,i)

In [0]:
print(pred[:12,:])

In [0]:
y_te[4]

In [0]:
plt.plot(X_te[4])

In [0]:
print(np.sum(pred[4], axis=0))

In [0]:
print(np.min(pred[:,1]))

In [0]:
pred[:,1] -= np.min(pred[:,1])

In [0]:
from sklearn.preprocessing import normalize

new_vec = pred[:,1:]
normalized = new_vec/np.linalg.norm(new_vec)
print(normalized[:12])

In [0]:
output = []
for i in pred:
  if i[2] > i[1]:
    output.append(2)
  if i[2] <= i[1]:
    output.append(1)
error = np.absolute(np.array(output)-label[1800:])
print(np.sum(error))

In [0]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(output, label[1800:])

In [0]:
confusion

In [0]:
import seaborn as sns
df_cm = pd.DataFrame(confusion, index = [i for i in "AB"],
                  columns = [i for i in "AB"])

df_cm_norm = df_cm/df_cm.sum(axis=1)

plt.figure(figsize = (10,7))
sns.heatmap(df_cm_norm, annot=True)

In [0]:
for k,i in enumerate(output):
  if i == 2:
    print(k,i)

In [0]:
pred_train = model.predict(X_train)

In [0]:
print(y_train[0:12])

In [0]:
pred_train[0:12]